In [ ]:
import fitz # import PyMuPDF
from pathlib import Path
import os
import re
import pandas as pd

# テキストを置換：項目名を△△で囲む
def text_replace(text):

   # text = re.sub(r'\n([^\d\.|^[A-Z])', r'\1', text)  # 不要な改行の削除
   text = re.sub(r'\n([\d\.]{2,}[A-Z]*)', r'\n△\1△', text)
   text = re.sub(r'^([\d\.]{2,})', r'\n△\1△', text) # △0.1△ 
   text = re.sub(r'([^A-Z][A-Z]\d[\.\d]*)', r'\n△\1△', text) # △B2.2△  
   text = re.sub(r'→ \n△(.+)△', r'→\1', text) # (→ \n△
   text = re.sub(r' \n△(.+)△ (から)\n△(.+)△', r'\1\2\3', text)
   text = re.sub(r'^(第.+?章)', r'\1△', text)
   text = re.sub(r'△(\d\d\d)△', r'\1', text)
   return text

# pymupdfでPDFをテキストに変換
def pdf2text(filepath):
   basename_without_ext = os.path.splitext(os.path.basename(filepath))[0]
   print(basename_without_ext)

   doc = fitz.open(filepath)  # open a supported document

   text = ""
   # text = chr(12).join([page.get_text() for page in doc])
   text = "".join([page.get_text() for page in doc])
   text = text_replace(text)
   return text, basename_without_ext

def text2df(text):
   # split()で文字列を分割して取得したリストにstrip()を適用する。
   # 空白を含むカンマ区切り文字列の余分な空白を除去してリスト内包表記でリスト化
   # list_text = [x.strip() for x in text[0].split('△')]
   list_text = [x.strip() for x in text.split('△')]
   list_text = [a for a in list_text if a != '']  # 内包表記で空の要素を駆逐する
   # print(list_text)

   # キーとバリューが交互に並ぶリストを辞書にする
   dict_text = dict(zip(list_text[0::2], list_text[1::2]))
   #print(dict_text)

   # 異なる長さのリストを含む辞書をpd.DataFrameに変換
   df2 = pd.DataFrame.from_dict(dict_text, orient='index')
   # df2
   df2.to_excel('pndas_to_excel.xlsx', sheet_name='new_sheet_name')
   return df2


filepath = r"pdf/CM14.pdf"
text = pdf2text(filepath)
# 戻り値の型はタプルとなる。text, basename_without_ext
dataframe = text2df(text[0])

# path_w = r"cm0.txt"
other_filepath = os.path.join(os.path.dirname(filepath), text[1]) # basename_without_ext)
other_ext_filepath = other_filepath + '.txt'

with open(other_ext_filepath, mode='w') as f:
    f.write(text[0])

In [ ]:
dataframe

In [ ]:
import glob
files = glob.glob("pdf/*.pdf")
print(files)

# 旧（現行）CM

In [ ]:
# https://qiita.com/poorko/items/9140c75415d748633a10
# Pythonのスクレイピングで文字だけを抜き出す汎用的な方法
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

# urlを指定してBeautifulSoupオブジェクトを返す
def scrape_html(url):
  html = requests.get( url ) # .text
  soup = BeautifulSoup(html.content.decode("utf-8", "ignore"), "html.parser")
  # print(soup.prettify)
  return soup

# BeautifulSoupオブジェクトを引数に、テキストを返す
def get_text(soup):
  # scriptやstyleを含む要素を削除(decompose)する
  for script in soup(["script",  "style"]):
      script.decompose()
  # print(soup)
  # タグを削除してテキストを取得
  text = soup.get_text()
  # print(text)
  # textを改行ごとにリストに入れて、リスト内の要素の前後の空白を削除(strip)、内包表記でリスト化
  lines = [line.strip() for line in text.splitlines()]
  #lines = []
  # for line in text.splitlines():
  #  lines.append(line.strip())
  # print(lines)

  text = "\n".join(line for line in lines if line)
  # print(text)
  return text

# URLを引数に、リンクのリストを出力
def list_links(url):
    parse_html = scrape_html(url)
    title_lists = parse_html.find_all("a")

    # hrefを取得してリスト化
    name_list = []
    url_list = []
    for i in title_lists:
        name_list.append(i.string)
        url_list.append(i.get('href'))   # (i.attrs["href"]) getはなければNoneが返ってきます

    # リストの空の要素を消す
    # [x for x in url_list if x != '']
    url_list_b = [x for x in url_list if x is not None]
    # 先頭が数字のリンクを抽出し、リスト化
    url_list_b = [x for x in url_list_b if re.match('^\d', x)]
    # print(url_list_b)
    
    return url_list_b

# URLを引数にurlをパースして、リンクのリストを返す
def url_parse(access_url):
   parsed_url = urlparse(access_url) 
   path_list = parsed_url.path.split("/")[-2:]
   # フォーマットする 
   # url = '{uri.scheme}://{uri.netloc}/path_list[0]/'.format(uri=urlparse(access_url))
   url_d = parsed_url.scheme + '://' + parsed_url.netloc + '/' + path_list[0] + '/'
   print(url_d)

   # リンク先をリスト化
   url_list_b = list_links(access_url)
   # url_full = url_d + url_list_b[0]
   # htmlファイルをスクレイプする
   # print(scrape_html(url_d + url_list_b[0])) #.decode("utf-8")
   # html_text = get_text(scrape_html(url_full))

   # urlのリストから、url_fullをリスト化
   url_full = [url_d + i for i in url_list_b]
   """
   url_full = []
   for i in url_list_b:
      url_full.append(url_d + i)
      # html_text += get_text(scrape_html(url_full))
   """
   return url_full

# テスト用
get_text(scrape_html("https://qiita.com/poorko/items/9140c75415d748633a10"))
# list_links("https://catill.bitbucket.io/CM/mokuji.html")


'Pythonのスクレイピングで文字だけを抜き出す汎用的な方法 #Python3 - Qiita\nsearchsearchSearchLoginSignupsearchTrendQiita Engineer Festa 2024QuestionOfficial EventOfficial ColumnsignpostCareerOrganization\n138Go to list of users who liked144more_horizinfoMore than 1 year has passed since last update.@poorkoPythonのスクレイピングで文字だけを抜き出す汎用的な方法PythonscrapingPython3BeautifulSoupGoogleColaboratoryLast updated at 2022-03-21Posted at 2018-12-16\nPythonのスクレイピングで文字だけきれいに抽出する汎用的な方法\n初投稿です。\npythonのスクレイピングで複数のサイトから文字だけ全部抜き出そうとする際に、うまく機能する汎用的なやり方を理解するのにわりと時間がかかったため、ここにしたためておきます。\nまず、英語でどんぴしゃり回答していらっしゃる方がいたので、英語いける方はこちらのURLを見ていただければ試合終了です。\nExtracting text from HTML file using Python\n私はコードを調べつつでないと理解できないPrograming初心者で、割と処理の理解に時間がかかったので、やり方を０から簡単に解説します。必要なところだけさくっと見てください。\n事前準備\nインストール\nimport requests\nimport pandas as pd\nfrom bs4 import BeautifulSoup\nHTML取得\nhtml=requests.get("https://www.updateurself.com/2018/11/14/the-way-to-challenge/").text\nsoup=BeautifulSoup(html,"html.parser")\n#print(soup.prettify)\nちなみにこのURLは私がやっている

In [ ]:
access_url = "https://catill.bitbucket.io/CM/mokuji.html"
url_full = url_parse(access_url)
# print(url_full)
for i in url_full:
    html_text += get_text(scrape_html(i))



https://catill.bitbucket.io/CM/


In [ ]:
# print(html_text)
import os

path = os.getcwd()
print(path)
other_filepath = os.path.join(os.path.dirname(path), 'html_text') # basename_without_ext)
print(other_filepath)
other_ext_filepath = other_filepath + '.txt'
print(other_ext_filepath)


with open(other_ext_filepath, mode='w') as f:
    f.write(html_text)


c:\Users\onowa\pumupdf
c:\Users\onowa\html_text
c:\Users\onowa\html_text.txt
